## Tugas Besar II IF3170 Inteligensi Buatan
## Heart Disease Classification

#### Kelompok 18 - Nira Model Kami:
- Nira Rizki Ramadhani / 13516018
- Sinaga Yoko Christoffel / 13516052
- Haifa Fadhila Ilma / 13516076
- Nadija Herdwina Putri Soerojo / 13516130
- Muhammad Fadhriga Bestari / 13516154

In [9]:
import pandas as pd
import numpy as np
import graphviz
import pickle
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import itertools
import random
import sklearn.exceptions
import warnings
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

from sklearn.datasets import load_iris 
from sklearn.model_selection import train_test_split, KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier 
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn import neighbors, datasets
from sklearn.externals import joblib 

### Membaca Dataset Heart Disease

In [10]:
train_data = pd.read_csv('tubes2_HeartDisease_train.csv')
train_data

Column1  Column2  Column3 Column4 Column5 Column6 Column7 Column8  \
0         54        1        4     125     216       0       0     140   
1         55        1        4     158     217       0       0     110   
2         54        0        3     135     304       1       0     170   
3         48        0        3     120     195       0       0     125   
4         50        1        4     120       0       0       1     156   
5         64        0        4     130     303       0       0     122   
6         63        1        4     130     308       0       0     138   
7         58        1        2     130     251       0       0     110   
8         42        1        2     150     268       0       0     136   
9         54        1        3     120     258       0       2     147   
10        58        1        4     115       0       ?       0     138   
11        49        1        3     118     149       0       2     126   
12        61        1        1     142     200       1       1     100   
13        55        0        2     132     342       0       0     166   
14        60        1        4     132     218       0       1     140   
15        67        1        4       ?     369       0       0       ?   
16        62        1        3     160       0       0       0      72   
17        57        1        4     150     276       0       2     112   
18        43        1        1     120     291       0       1     155   
19        67        1        4     125     254       1       0     163   
20        59        1        4     110       0       ?       0      94   
21        45        0        2     112     160       0       0     138   
22        51        1        3     135     160       0       0     150   
23        53        1        4     120     246       0       0     116   
24        51        0        3     120     295       0       2     157   
25        53        1        3     155     175       1       1     160   
26        64        0        4     180     325       0       0     154   
27        52        1        4     108     233       1       0     147   
28        41        1        4     104       0       0       1     111   
29        55        1        4     140     201       0       0     130   
..       ...      ...      ...     ...     ...     ...     ...     ...   
749       49        0        3     130     207       0       1     135   
750       32        1        2     110     225       0       0     184   
751       57        1        2     154     232       0       2     164   
752       74        1        4     155     310       0       0     112   
753       59        1        3     150     212       1       0     157   
754       53        1        4     130       0       0       2     135   
755       35        1        2     122     192       0       0     174   
756       34        0        2     130     161       0       0     190   
757       55        1        3       ?       ?       0       1       ?   
758       54        0        3     110     214       0       0     158   
759       63        1        4     136       0       0       0      84   
760       37        1        3     118     240       0       2     165   
761       56        1        4     150     230       0       1     124   
762       53        1        2     120       0       0       0      95   
763       50        1        3     129     196       0       0     163   
764       54        0        2     120     230       1       0     140   
765       56        1        4     120       0       0       1     100   
766       58        1        4     125     300       0       2     171   
767       43        1        4     132     247       1       2     143   
768       59        1        4     120       0       0       0     115   
769       72        1        4     160       ?       1       2     130   
770       52        1        4     160     246       0       1      82   
771      

#### Drop Insignificant Columns
Pada data diatas, terdapat *missing values* '?' dan beberapa kolom dengan jumlah *missing values* yang terlalu banyak akan di-drop karena data didalamnya kurang mempengaruhi proses prediksi. Berikut penghitungan persentase missing values di 3 kolom terakhir, didapat persentase yang cukup tinggi pada kolom 11, 12 dan 13:

In [11]:
count = 0

for data in train_data['Column11'] :
    if (data == '?') :
        count += 1
print "Missing data percentage in column 11:",round(float(count)/779*100,2),"%" 
count = 0

for data in train_data['Column12'] :
    if (data == '?') :
        count += 1
print "Missing data percentage in column 12:",round(float(count)/779*100,2),"%" 
count = 0

for data in train_data['Column13'] :
    if (data == '?') :
        count += 1
print "Missing data percentage in column 13:",round(float(count)/779*100,2),"%" 

Missing data percentage in column 11: 33.63 %
Missing data percentage in column 12: 65.98 %
Missing data percentage in column 13: 52.37 %


### Replace Missing Values
Dibawah ini merupakan algoritma untuk mengganti *missing values* dengan modus atau median dari data kolom tersebut. Data yang bersifat *categorical* akan menggunakan modus dan yang berupa angka ukuran menggunakan median.

Selain itu dilakukan pula *dropping* pada kolom yang kurang signifikan. Pada kasus ini dibagi 2, train data X1 dan X2 dimana pada X1 kolom 11 diikutkan pada data (hanya kolom 12 dan 13 yang di-*drop* dan pada X2 kolom 11 juga di-*drop*.

In [12]:
# Replace all missing values to None
train_data.replace({'?' : None, 'None' : None}, inplace=True)

# Replace missing values per column
train_data['Column4'].fillna(value = train_data['Column4'].median(),inplace=True)
train_data['Column5'].fillna(value = train_data['Column5'].median(),inplace=True)
train_data['Column6'].fillna(value = train_data['Column6'].mode()[0],inplace=True)
train_data['Column7'].fillna(value = train_data['Column7'].mode()[0],inplace=True)
train_data['Column8'].fillna(value = train_data['Column8'].median(),inplace=True)
train_data['Column9'].fillna(value = train_data['Column9'].mode()[0],inplace=True)
train_data['Column10'].fillna(value = train_data['Column10'].median(),inplace=True)
train_data['Column11'].fillna(value = train_data['Column11'].mode()[0],inplace=True)
train_data['Column12'].fillna(value = train_data['Column12'].mode()[0],inplace=True)
train_data['Column13'].fillna(value = train_data['Column13'].mode()[0],inplace=True)

train_data_X1 = train_data.drop(["Column14","Column12","Column13"], axis=1)
train_data_X2 = train_data.drop(["Column14","Column12","Column13","Column11"], axis=1)
train_data_y = train_data["Column14"]

print "Hasil replacement (contoh pada tabel X1):"
train_data_X1

Hasil replacement (contoh pada tabel X1):


Column1  Column2  Column3 Column4 Column5 Column6 Column7 Column8  \
0         54        1        4     125     216       0       0     140   
1         55        1        4     158     217       0       0     110   
2         54        0        3     135     304       1       0     170   
3         48        0        3     120     195       0       0     125   
4         50        1        4     120       0       0       1     156   
5         64        0        4     130     303       0       0     122   
6         63        1        4     130     308       0       0     138   
7         58        1        2     130     251       0       0     110   
8         42        1        2     150     268       0       0     136   
9         54        1        3     120     258       0       2     147   
10        58        1        4     115       0       0       0     138   
11        49        1        3     118     149       0       2     126   
12        61        1        1     142     200       1       1     100   
13        55        0        2     132     342       0       0     166   
14        60        1        4     132     218       0       1     140   
15        67        1        4     130     369       0       0     140   
16        62        1        3     160       0       0       0      72   
17        57        1        4     150     276       0       2     112   
18        43        1        1     120     291       0       1     155   
19        67        1        4     125     254       1       0     163   
20        59        1        4     110       0       0       0      94   
21        45        0        2     112     160       0       0     138   
22        51        1        3     135     160       0       0     150   
23        53        1        4     120     246       0       0     116   
24        51        0        3     120     295       0       2     157   
25        53        1        3     155     175       1       1     160   
26        64        0        4     180     325       0       0     154   
27        52        1        4     108     233       1       0     147   
28        41        1        4     104       0       0       1     111   
29        55        1        4     140     201       0       0     130   
..       ...      ...      ...     ...     ...     ...     ...     ...   
749       49        0        3     130     207       0       1     135   
750       32        1        2     110     225       0       0     184   
751       57        1        2     154     232       0       2     164   
752       74        1        4     155     310       0       0     112   
753       59        1        3     150     212       1       0     157   
754       53        1        4     130       0       0       2     135   
755       35        1        2     122     192       0       0     174   
756       34        0        2     130     161       0       0     190   
757       55        1        3     130     225       0       1     140   
758       54        0        3     110     214       0       0     158   
759       63        1        4     136       0       0       0      84   
760       37        1        3     118     240       0       2     165   
761       56        1        4     150     230       0       1     124   
762       53        1        2     120       0       0       0      95   
763       50        1        3     129     196       0       0     163   
764       54        0        2     120     230       1       0     140   
765       56        1        4     120       0       0       1     100   
766       58        1        4     125     300       0       2     171   
767       43        1        4     132     247       1       2     143   
768       59        1        4     120       0       0       0     115   
769       72        1        4     160     225       1       2     130   
770       52        1        4     160     246       0       1      82   
771      

### Split Data dengan K-Fold

In [13]:
def k_fold_train(k, model_type, X, y) :
    kf = KFold(k, shuffle=True)
    model = None
    # Container
    accuracies = []
    precisions = []
    recalls = []
    
    if model_type == 'nb' :
        model = GaussianNB()
    elif model_type == 'tree' :
        model = DecisionTreeClassifier()
    elif model_type == 'knn' :
        model = KNeighborsClassifier(n_neighbors=15)
    else :
        model = MLPClassifier(max_iter=1000)
    
    for train_idx, test_idx in kf.split(X) :
        x_train, x_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        model.fit(x_train, y_train)
        
        accuracies.append(accuracy_score(y_test, model.predict(x_test)))
        precisions.append(precision_score(y_test, model.predict(x_test), average = 'macro'))
        recalls.append(recall_score(y_test, model.predict(x_test), average = 'macro'))
        
        avg_accuracies = np.mean(accuracies)
#         avg_precisions = np.mean(precisions)
#         avg_recalls = np.mean(recalls)
        return avg_accuracies

def k_fold_train_model(k, model_type, X, y) :
    kf = KFold(k, shuffle=True)
    model = None
    # Container
    accuracies = []
    precisions = []
    recalls = []
    
    if model_type == 'nb' :
        model = GaussianNB()
    elif model_type == 'tree' :
        model = DecisionTreeClassifier()
    elif model_type == 'knn' :
        model = KNeighborsClassifier(n_neighbors=15)
    else :
        model = MLPClassifier(max_iter=1000)
    
    for train_idx, test_idx in kf.split(X) :
        x_train, x_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        model.fit(x_train, y_train)
        
        accuracies.append(accuracy_score(y_test, model.predict(x_test)))
        precisions.append(precision_score(y_test, model.predict(x_test), average = 'macro'))
        recalls.append(recall_score(y_test, model.predict(x_test), average = 'macro'))
        
        avg_accuracies = np.mean(accuracies)
#         avg_precisions = np.mean(precisions)
#         avg_recalls = np.mean(recalls)
        return model

#### Naive Bayes Model
Model naive bayes pada train_data_X1 dan train_data_X2, lalu dihitung rata-rata akurasi keduanya

In [14]:
i = 0
total1 = 0
total2 = 0
count = 10
for i in range(count) :
    avg_accuracy1 = k_fold_train(10, 'nb', train_data_X1.values, train_data_y.values)
    total1 += avg_accuracy1
    avg_accuracy2 = k_fold_train(10, 'nb', train_data_X2.values, train_data_y.values)
    total2 += avg_accuracy2

avg_accuracy1 = total1 / count    
print "Naive bayes average accuracy 1 after 10 tries : ", avg_accuracy1 
avg_accuracy2 = total2 / count    
print "Naive bayes average accuracy 2 after 10 tries : ", avg_accuracy2 

Naive bayes average accuracy 1 after 10 tries :  0.5461538461538462
Naive bayes average accuracy 2 after 10 tries :  0.5846153846153845


#### Decision Tree Model
Model decision tree pada train_data_X1 dan train_data_X2, lalu dihitung rata-rata akurasi keduanya

In [15]:
i = 0
total1 = 0
total2 = 0
count = 10
for i in range(count) :
    avg_accuracy1 = k_fold_train(10, 'tree', train_data_X1.values, train_data_y.values)
    total1 += avg_accuracy1
    avg_accuracy2 = k_fold_train(10, 'tree', train_data_X2.values, train_data_y.values)
    total2 += avg_accuracy2

avg_accuracy1 = total1 / count    
print "Decision Tree average accuracy 1 after 10 tries : ", avg_accuracy1 
avg_accuracy2 = total2 / count    
print "Decision Tree average accuracy 2 after 10 tries : ", avg_accuracy2 

Decision Tree average accuracy 1 after 10 tries :  0.4807692307692307
Decision Tree average accuracy 2 after 10 tries :  0.4564102564102564


#### K Nearest Neighbors
Model kNN (neighbor: 15) pada train_data_X1 dan train_data_X2, lalu dihitung rata-rata akurasi keduanya

In [16]:
i = 0
total1 = 0
total2 = 0
count = 10
for i in range(count) :
    avg_accuracy1 = k_fold_train(10, 'knn', train_data_X1.values, train_data_y.values)
    total1 += avg_accuracy1
    avg_accuracy2 = k_fold_train(10, 'knn', train_data_X2.values, train_data_y.values)
    total2 += avg_accuracy2

avg_accuracy1 = total1 / count    
print "kNN average accuracy 1 after 10 tries : ", avg_accuracy1 
avg_accuracy2 = total2 / count    
print "kNN average accuracy 2 after 10 tries : ", avg_accuracy2 

kNN average accuracy 1 after 10 tries :  0.49102564102564106
kNN average accuracy 2 after 10 tries :  0.49230769230769234


#### Multilayer Perceptron
Model MLP pada train_data_X1 dan train_data_X2, lalu dihitung rata-rata akurasi keduanya

In [17]:
i = 0
total1 = 0
total2 = 0
count = 10
for i in range(count) :
    avg_accuracy1 = k_fold_train(10, 'mlp', train_data_X1.values, train_data_y.values)
    total1 += avg_accuracy1
    avg_accuracy2 = k_fold_train(10, 'mlp', train_data_X2.values, train_data_y.values)
    total2 += avg_accuracy2

avg_accuracy1 = total1 / count    
print "MLP average accuracy 1 after 10 tries : ", avg_accuracy1 
avg_accuracy2 = total2 / count    
print "MLP average accuracy 2 after 10 tries : ", avg_accuracy2

MLP average accuracy 1 after 10 tries :  0.49615384615384617
MLP average accuracy 2 after 10 tries :  0.49615384615384617


### Model Terbaik:

Berdasarkan perhitungan rata-rata akurasi dari keempat model diatas, didapatkan model ***Naive Bayes*** yang memiliki rata-rata akurasi tertinggi, dan penggunaan data awal dilakukan *dropping* pada kolom 12 dan kolom 13 (train_data_X1).

#### Menyimpan Model Terbaik ke File

In [18]:
model = k_fold_train_model(10, 'nb', train_data_X1.values, train_data_y.values)
pickle.dump(model, open('model.sav', 'wb'))
joblib.dump(model, 'model.joblib')

['model.joblib']

### Prediksi Data Test
Dibawah ini akan dilakukan prediksi data test dengan model terbaik yang sudah ada.

In [19]:
# Load Model
model = pickle.load(open("model.sav", "rb"))

Load file data test, me-*replace missing values* dan melakukan *dropping* pada kolom 12 dan 13.

In [20]:
# Load File
test_data = pd.read_csv('tubes2_HeartDisease_test.csv')
test_data

# Replace all missing values to None
test_data.replace({'?' : None, 'None' : None}, inplace=True)

# Replace missing values per column
test_data['Column4'].fillna(value = test_data['Column4'].median(),inplace=True)
test_data['Column5'].fillna(value = test_data['Column5'].median(),inplace=True)
test_data['Column6'].fillna(value = test_data['Column6'].mode()[0],inplace=True)
test_data['Column7'].fillna(value = test_data['Column7'].mode()[0],inplace=True)
test_data['Column8'].fillna(value = test_data['Column8'].median(),inplace=True)
test_data['Column9'].fillna(value = test_data['Column9'].mode()[0],inplace=True)
test_data['Column10'].fillna(value = test_data['Column10'].median(),inplace=True)
test_data['Column11'].fillna(value = test_data['Column11'].mode()[0],inplace=True)

test_data_X = test_data.drop(["Column12","Column13"], axis=1)

Prediksi model pada data test

In [21]:
prediction = model.predict(test_data_X)
print "Hasil Prediksi:"
count = 0
for pre in prediction:
    count+=1
    print "Data test",count,":",pre

Hasil Prediksi:
Data test 1 : 0
Data test 2 : 0
Data test 3 : 1
Data test 4 : 1
Data test 5 : 0
Data test 6 : 0
Data test 7 : 4
Data test 8 : 1
Data test 9 : 0
Data test 10 : 1
Data test 11 : 1
Data test 12 : 1
Data test 13 : 0
Data test 14 : 0
Data test 15 : 0
Data test 16 : 3
Data test 17 : 2
Data test 18 : 0
Data test 19 : 0
Data test 20 : 0
Data test 21 : 0
Data test 22 : 1
Data test 23 : 0
Data test 24 : 1
Data test 25 : 3
Data test 26 : 0
Data test 27 : 1
Data test 28 : 3
Data test 29 : 3
Data test 30 : 1
Data test 31 : 0
Data test 32 : 3
Data test 33 : 2
Data test 34 : 0
Data test 35 : 1
Data test 36 : 0
Data test 37 : 0
Data test 38 : 0
Data test 39 : 0
Data test 40 : 0
Data test 41 : 1
Data test 42 : 0
Data test 43 : 0
Data test 44 : 1
Data test 45 : 3
Data test 46 : 1
Data test 47 : 1
Data test 48 : 2
Data test 49 : 1
Data test 50 : 1
Data test 51 : 1
Data test 52 : 0
Data test 53 : 2
Data test 54 : 1
Data test 55 : 0
Data test 56 : 1
Data test 57 : 0
Data test 58 : 1
Data te